# HPE Redfish Ansible playbooks

Version 0.129

## Introduction

The goal of this notebook is to present another approach for creating Redfish Ansible Playbooks compared to the approach used in the previous notebook.

The HPE Ansible Redfish [Gitub repository](https://github.com/HewlettPackard/ansible-ilorest-role) proposes three sets of Ansible Redfish Playbooks examples using:

  * the iLOrest command
  * the Redfish Galaxy collection
  * Ansible modules derived from the [examples](https://github.com/HewlettPackard/python-ilorest-library/tree/master/examples/Redfish) of the `python-ilorest-library`.

In this Jupyter Notebook, you will study an [Redfish Ansible Playbook](3/set_uid_light.yml) based upon the [`set_uid_light.py`](https://github.com/HewlettPackard/python-ilorest-library/tree/master/examples/Redfish/set_uid_light.py) example of the the HPE [`python-ilorest-library`](https://github.com/HewlettPackard/python-ilorest-library/tree/master/). This python example has been modified to become an [Ansible module](https://github.com/HewlettPackard/ansible-ilorest-role/blob/master/library/set_uid_light.py). 

## <a id = "EnvironmentPreparation">Environment preparation</a>

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [ ]:
########## Environment preparation (Version: 0.132) ############

# Set Student ID number
stdid={{ STDID }}
Id=$(id --user --name)
NbId=3
InvFile=${NbId}/hosts

# location and ports variables
IloSyBasePort={{ ILOSYBASEPORT }}
let iLO5SimulatorBasePort=$IloSyBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${stdid}

iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

# Fake Credentials as we are testing against a BMC simulator
OvSsoToken="FakeOvSsoToken"

# Miscellaneous
WorkshopDir=$PWD
HpePythonRedfishVenv="${NbId}/HpePythonRedfishVenv"
export PYTHONPATH="${WorkshopDir}/${NbId}/library/"
w=$(basename $PWD)

alias ResetSimulators="../create-globalbmc.shc.x &>/dev/null; sleep 1"

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &>/dev/null && 
        echo -e "\n\tGood News: $bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done


# Create the Ansible inventory file
cat > ${InvFile} << __EOF__
[OneViewManagedBmcs]
${iLO5SimulatorIP} ansible_port=${iLO5SimulatorPort}

[OneViewManagedBmcs:vars]
ansible_python_interpreter=${WorkshopDir}/${HpePythonRedfishVenv}/bin/python3
ansible_search_path=${HpePythonRedfishVenv}

# Below is a fake session token as we are testing against an iLO 5 simulator.
# In real life, you should populate this variable with the token obtained
# lab 1 of this workshop.
token="${OvSsoToken}"
__EOF__


## Virtual Python environment creation

In order to completely isolate this notebook environment from other notebooks or student python environments, it is safer to create your dedicated Python virtual environment.

> NOTE: This `Venv` creation can take up to **2 minutes**. Just wait until message `Finished creating Venv` is diplayed

In [ ]:
# Create Virtual Python environment (Venv) [Version 0.111]
[ -d ${HpePythonRedfishVenv} ] && rm -r ${HpePythonRedfishVenv} &>/dev/null 
python3 -m venv ${HpePythonRedfishVenv}                         &>/dev/null
source ${HpePythonRedfishVenv}/bin/activate                     &>/dev/null
PS1="[PEXP\[\]ECT_PROMPT>"                                      # Avoid Venv long prompt messing up outputs

# Populate Python Venv with the HPE python-ilorest-library
pip install wheel                                               &>/dev/null
pip install python-ilorest-library                              &>/dev/null

# Install latest Ansible in the Venv
pip install jmespath                                            &>/dev/null
pip install ansible                                             &>/dev/null

echo -e "\n\n\tFinished creating Venv\n\n"

### <a id = "iLO5SimulatorRestart">Restart iLO 5 simulator</a>

If you need or desire to restart your iLO 5 simulator to restart this workshop from scratch or for other reasons, run the following cell at any time.

In [ ]:
# iLO 5 Simulator restart
ResetSimulators 

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

## Get and set Redfish properties using HPE's python-ilorest-library

In the previous notebook, to discover the `Chassis` collection and the value of the indicator LEDs, you had to crawl the Redfish tree using Ansible and its built-in `uri` module.

When using a custom Ansible module derived from a `python-ilorest-library` example, the Redfish tree crawling is performed inside the Ansible module. 

HPE `python-ilorest-library` has been loaded in the Python Venv of the second cell of this notebook. It will allow the creation of a redfish object containing authentication token as well as GET and SET methods.

You will find your custom Ansible modules and other python files in your `library` sub-folder.

### Indicator LED

TBD

A convenient way to study the playbook of the next cell is to open it in a different view in this pane. Right click on this Notebook tab name and select `New View for Notebook` to open a new view:
<img src="Pictures/3-NewViewForNotebook.png" align=middle alt="New View for Notebook" height=30px />

Then, click on this [file link](3/SetIndicatorLEDUsingiLOrestLibrary.yml). You can study as well the Ansible Python module [source file](3/library/set_uid_light.py).

If you need more space, type `Ctrl-B` to hide the left pane. You can have it reappear by hitting `Ctrl-B` again.

In [ ]:
# Launch Ansible Playbook to toggle Indicator LED
ansible-playbook -i ${InvFile} ${NbId}/SetIndicatorLEDUsingiLOrestLibrary.yml

## Summary

In this workshop, you used an Ansible module created with the HPE `python-ilorest-library` to get and set the same Redfish resources as in previous notebook. The advantage of this method is to move the complexity of the Redfish tree crawling process into a python module that can use a dedicated Redfish library for authentication and error handling.

You are now ready to go through the [next Notebook](4-GalaxyModulesAndDmtfPlaybooks.ipynb).